# Swarm Simulator for Outer Space

This swarm simulator will explore different swarm algorithms for an application in outer space. This approach will divide a plot into grid squares, which can then be analysed to determine and control the behaviour of each individual agent in the swarm.

In [ ]:
#imports
import sympy as sym
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import deque
from scipy.spatial.distance import cdist, pdist, euclidean
from sympy import sin, cos, symbols, lambdify

from robot import Robot
from dust_devil import DustDevil

import random
import datetime
import os
import math

import Processing_Functions

app = dash.Dash(__name__)

In [ ]:
#Plotly offline mode
init_notebook_mode(connected=True)

## Initialisation of Constants

In [ ]:
#initialising parameters
experiment = False
conversion_ms = 1/3.6 #km/h to m/s conversion actor
timer = 0


#Robot Parameters
robot_number = 50
robot_speed = 13*conversion_ms #m/s
robot_radius = 50 
robot_mass = 20 #kg
x_robot = []
y_robot = []
cluster_average = []
min_neighbours = []
gen_random = 0 #the random angle

#dust devil parameters
dust_time = 90 #time in seconds the dust devil will survive 
dust_speed = 150*conversion_ms #m/s
dust_radius = 100
dust_timer_count = 0
dust_devils = []
detection_range = 20
interception_time = []

#Area Parameters
area = 1 #area in km^2 
side = 1 #each side length in m
per_square_km = 100/(60*60*24)
probability_dust = (per_square_km*area) #number of dust devils per second

#Sensor Parameters
sensor_radius = 40000

#Mean Squared Distance
MSD = []

In [ ]:
print(probability_dust)

## Physics Constants

In [ ]:
#setting physics constants
G = 10392
power = 2
R = 200
max_force = 1
multiply = 1.5


In [ ]:
#Function to initialise robots
def initialise(robot_number,length):
    '''
    Returns the list of robot objects

            Parameters:
                    robot_number (int): Number of robots
                    length (int): Length of one side of the area

            Returns:
                   robots (list): List of robot objects
    '''
    robots = []
    X,Y = random_position(robot_number,-10,10)
    cnt = 0
    for x,y in zip(X,Y):
        walk_time = int(np.random.normal(5,3,1))
        heading = random.uniform(0,2*math.pi)
        if(cnt == 0):
            identifier = 0
        else:
            identifier = 1
            cnt = -1
        robots.append(Robot(x,y,1,0,0,robot_speed, walk_time,heading,identifier))
        cnt = cnt+1
    return robots

In [ ]:
#Function to return randomised x,y position array
def random_position(robot_number,start,finish):
    '''
    Returns random position array of x and y

            Parameters:
                    robot_number (int): Number of robots
                    length (int): Length of one side of the area

            Returns:
                   x (integer list): List of random x positions
                   y (integer list): List of random y positions
    '''
    
    x = []
    y = []
    for i in range(int(robot_number)):
        x.append(random.randint(start,finish))
        y.append(random.randint(start,finish))
        walk_time = int(np.random.normal(20,5,1))
    return x,y

In [ ]:
#Function to return square grid of x,y positions
def grid_center(length):
    '''
    Returns square grid position array of x and y

            Parameters:
                    length (int): Length of one side of the area

            Returns:
                   x (integer list): List of x positions in the grid
                   y (integer list): List of y positions in the grid
    '''
    division = length/6
    center_x = []
    center_y = []
    for i in range(1,6):
        for j in range(1,6):
            center_x.append(division*j)
            center_y.append(division*i)
    return center_x,center_y

In [ ]:
#Function to retrieve class object positions
def positions(objects):
    '''
    Returns 2D positions of class objects

            Parameters:
                    objects (list): List of objects

            Returns:
                   x (numpy array int): List of x positions of the object
                   y (numpy array int): List of y positions of the object
    '''
    x = []
    y = []
    for i in range(len(objects)):
       x.append(objects[i].x)
       y.append(objects[i].y) 
    return np.array(x),np.array(y)

In [ ]:
#Function to check if the robot has gone over the edge and changing the value to negative if it has
def bounce(x_updated,y_updated,x_change,y_change,length):
    '''
    Returns bounced x and y position

            Parameters:
                    x_updated (float): X position after movement update
                    y_updated (float): Y position after movement update
                    x_change (float):  Change from previous X position
                    y_change (float):  Change from previous Y position
                    length (float):    Length of one side of the area

            Returns:
                   x_updated (float): Bounced X position
                   y_updated (float): Bounced Y position
    '''
    #if the x is larger than the side limit, then bounce off
    if(x_updated > length or x_updated < 0):
        x_updated = x_updated-2*x_change
    #if the y is larger than the side limit, then bounce off
    if(y_updated > length or y_updated < 0):
        y_updated = y_updated-2*y_change
    return x_updated,y_updated

In [ ]:
#Function check if there is any collisions between the robots (placeholder)
def collision_detection(swarm,dust_devils):
    positions_robot = np.array([positions(swarm)])
    position_dust = np.array([positions(dust_devils)])
    
    check = False
    return check

In [ ]:
#Function to work out the magnitude of a 2D vector
def magnitude(x,y):
    '''
    Finds the magnitude of 2 floats
    
            Parameters:
                   x (float): a float value
                   y (float): a float value
            
            Returns:
                   mag (np array): the resulting np array of magnitudes from the two given values

    '''
    mag = np.sqrt(np.square(x)+np.square(y))
    return mag

In [ ]:
#Function to work out a 2D unit vector
def unit(vector):
    '''
    Finds the unit vector of a passed vector
    
            Parameters:
                   Vector(2D numpy array): the 2D position vector
            
            Returns:
                   Unit(2D numpy array): the resulting unit vector from the given vector

    '''
    #print("Vector:", vector)
    mag = magnitude(vector[0],vector[1])
    #print("Magnitude", mag)
    
    if(vector[0] == 0):
        x = 0
    else:
        x = vector[0]/mag
    if(vector[1]==0):
        y = 0
    else:
        y = vector[1]/mag
    unit = np.array([x,y])


    
    return unit
    

In [ ]:
#Function to check if division by denominator is non zero, if it is zero, then the component is returned as zero
def division_check(numerator, denominator):
    '''
    Checks if the division is possible without zero division, if not set the component as zero
    
            Parameters:
                   Numerator (float): the numerator of the division
                 Denominator (float): the denominator of the division
            
            Returns:
                     Result (float): the result of the division

    '''

    if(denominator == 0):
        result = 0
    else:
        result = numerator/denominator
    
    return result


In [ ]:
"""##Function to call physics based area coverage algorithm
def physics_walk(swarm,G,power,R,max_force,multiply,min_neighbours,cluster_average,dust_devils,detection_range,interception_time):
    '''
    Updates the robot position based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
                   min_neighbours (float): the minimum neighbour distance array
                   cluster_average (float): the cluster size average array
            Returns:
                   None

    '''

    #print("Physics Called \n===============")
    #setting initial constants
    timestep = 1
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    minimum_distance = 0
    amount = len(swarm)
    
    #retrieving x and y positions
    x,y = positions(swarm)
    x_dust,y_dust = positions(dust_devils)
    count = 0
    
    
    cluster_total = 0
    
    
    #looping through each robot in the swarm
    collision_time = 0
    for robot in swarm:
        cluster_count = 1
        
        #initialising the forces as 0
        force = np.array([0,0])
        detected=False
        
        if(len(x_dust)>0):
            #finding distance between the robot and the dust devils 
            x_dust = x_dust - robot.x
            y_dust = y_dust - robot.y

            distance_dust = np.sqrt(np.square(x_dust)+np.square(y_dust))
            dust_sorted = np.sort(distance_dust)
            
            if(dust_sorted[0]<detection_range):
                collision_time = collision_time + 1
        """
            end = False
            while(count<len(dust_sorted) and end!=True):
                print(dust_sorted[count])
                if dust_sorted[count]<R:
                    detected = True
                    distance_unit = unit((x_dust[count],y_dust[count]))
                    force = force+(max_force*distance_unit)
                    print("Detected!")
                    end = True
                count = count+1
        """

        #finding the distance between the current robot and the other robots
        x_dist_unsorted = x-robot.x
        y_dist_unsorted = y-robot.y



        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))

        #calculating the minimum distance
        minimum_distance = minimum_distance + min(distance[np.nonzero(distance)])
        print("Min: ",minimum_distance)


        #determining the robots within 1.5R distance
        distance_local = (distance<multiply*R)


        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        force_change_dir = 0

        robot_number = len(distance_local)




        for i in range(len(x_dist)):

            #storing current position 
            current_position = np.array([position[0,i],position[1,i]])


            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])

            #checking if in the same cluster, if so adding to cluster count
            if(mag<0.2*R and mag!=0):
                cluster_count = cluster_count+1
            
            #force calculation only kicks in if dust devil not detected
            if(detected == False):
                #calculating the force
                numerator = (G*(robot.mass**2))
                denominator = mag**2


                #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
                force_change = division_check(numerator,denominator)

                #calculating the unit vector of the position
                distance_unit = unit(current_position)

                #if the magnitude is bigger than R, then a force is added to draw the robots together             
                if(mag>R):
                    force_change_dir = distance_unit

                #if the magnitude is smaller than R, then a force is added to push the robots apart 
                elif(mag<R):
                    force_change_dir = -distance_unit
                else:
                    force_change_dir = np.array([0,0])

                #calculating new force change based on direction and magnitude
                force_delta = force_change_dir*force_change

                #constraining force to the maximum
                if(force_delta[0] > max_force):
                    force_delta[0] = max_force
                elif(force_delta[0]<-max_force):
                    force_delta[0] = -max_force
                if(force_delta[1] > max_force):
                    force_delta[1] = max_force
                elif(force_delta[1]<-max_force):
                    force_delta[1] = -max_force

                #calculating new force
                force = force+force_delta
                
        cluster_total = cluster_total+cluster_count



        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy



        #keeping the velocity within the maximum velocity of the robot
        if(velocity_x>robot.max_velocity):
            velocity_x = robot.max_velocity
        if(velocity_y>robot.max_velocity):
            velocity_y = robot.max_velocity
        if(velocity_x<-robot.max_velocity):
            velocity_x = -robot.max_velocity
        if(velocity_y<-robot.max_velocity):
            velocity_y = -robot.max_velocity

        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change

        #updating the position
        robot.update_position(x_updated,y_updated)
        count = count+1
    
    
    avg_min_distance = minimum_distance/len(swarm)
    cluster_avg = cluster_total/robot_number
    cluster_average.append(cluster_avg)    
   
   
    min_neighbours.append(avg_min_distance)
    interception_time.append(sum(interception_time)+collision_time)"""

In [ ]:
#Function to call physics based area coverage algorithm
def physics_walk(swarm,G,power,R,max_force,multiply,min_neighbours,cluster_average,detection_range):
    '''
    Updates the robot position based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
                   min_neighbours (float): the minimum neighbour distance array
                   cluster_average (float): the cluster size average array
            Returns:
                   None

    '''

    #print("Physics Called \n===============")
    #setting initial constants
    timestep = 1
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    minimum_distance = 0
    amount = len(swarm)
    
    #retrieving x and y positions
    x,y = positions(swarm)
    x_dust,y_dust = np.array(positions(dust_devils))
    #print("X_dust: ", x_dust)
    #print(len(x_dust))
    count = 0
    
    
    cluster_total = 0
    
    #looping through each robot in the swarm
    collision_time = 0
    for i in range(len(swarm)):
        #robot
        robot = swarm[i]
        x_copy,y_copy = x.copy(),y.copy()
        x_copy.pop(i)
        y_copy.pop(i)
        cluster_count = 1
        
        #initialising the forces as 0
        force = np.array([0,0])
        
        if(len(x_dust)>0):
            #finding distance between the robot and the dust devils 
            x_dust = x_dust - robot.x
            y_dust = y_dust - robot.y

            distance_dust = np.sqrt(np.square(x_dust)+np.square(y_dust))
            dust_sorted = np.sort(distance_dust)
            
            if(dust_sorted[0]<detection_range):
                collision_time = collision_time + 1
                #print("Interception Time: ", collision_time)
        
        #finding the distance between the current robot and the other robots
        x_dist_unsorted = np.array(x_copy)-robot.x
        y_dist_unsorted = np.array(y_copy)-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))
        #print(len(distance))
        #print("Distances: ",distance)
        #calculating the minimum stance
        minimum_distance = minimum_distance + min(distance)
        #print("Minimum without Zero: ",min(distance))
        #[np.nonzero(distance)

        #determining the robots within 1.5R distance
        distance_local = np.nonzero(distance) and (distance<multiply*R)
        

        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        force_change_dir = 0
        
        robot_number = len(distance_local)
        
        
        
        
        for i in range(len(x_dist)):
            
            #storing current position 
            current_position = np.array([position[0,i],position[1,i]])

            
            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])
            
            #checking if in the same cluster, if so adding to cluster count
            if(mag<0.2*R and mag!=0):
                cluster_count = cluster_count+1
            
            #calculating the force
            numerator = (G*(robot.mass**2))
            denominator = mag**2

            
            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)
            
            #calculating the unit vector of the position
            distance_unit = unit(current_position)
            
            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>R):
                force_change_dir = distance_unit
            
            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])
       
            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if(force_delta[0] > max_force):
                force_delta[0] = max_force
            elif(force_delta[0]<-max_force):
                force_delta[0] = -max_force
            if(force_delta[1] > max_force):
                force_delta[1] = max_force
            elif(force_delta[1]<-max_force):
                force_delta[1] = -max_force
                

            #calculating new force
            force = force+force_delta
        cluster_total = cluster_total+cluster_count
        
        avg_min_distance = minimum_distance/len(swarm)


        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy
        

        
        #keeping the velocity within the maximum velocity of the robot
        if(velocity_x>robot.max_velocity):
            velocity_x = robot.max_velocity
        if(velocity_y>robot.max_velocity):
            velocity_y = robot.max_velocity
        if(velocity_x<-robot.max_velocity):
            velocity_x = -robot.max_velocity
        if(velocity_y<-robot.max_velocity):
            velocity_y = -robot.max_velocity
            
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)
        count = count+1
    cluster_avg = cluster_total/robot_number
    cluster_average.append(cluster_avg)    
   
   
    min_neighbours.append(avg_min_distance)
    return collision_time

In [ ]:
##Function to call physics based area coverage algorithm
def parameters(swarm,G,power,R,min_neighbours,cluster_average,dust_devils,detection_range,interception_time):
    '''
    Updates the robot position based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
                   min_neighbours (float): the minimum neighbour distance array
                   cluster_average (float): the cluster size average array
            Returns:
                   None

    '''

    #print("Physics Called \n===============")
    #setting initial constants
    timestep = 1
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    minimum_distance = 0
    amount = len(swarm)
    
    #retrieving x and y positions
    x,y = positions(swarm)
    x_dust,y_dust = positions(dust_devils)
    count = 0
    
    
    cluster_total = 0
    
    
    #looping through each robot in the swarm
    collision_time = 0
    for robot in swarm:
        cluster_count = 1
        


        #finding the distance between the current robot and the other robots
        x_dist_unsorted = x-robot.x
        y_dist_unsorted = y-robot.y



        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))

        #calculating the minimum distance
        minimum_distance = minimum_distance + min(distance[np.nonzero(distance)])
        print("Min: ",minimum_distance)


        #determining the robots within 1.5R distance
        distance_local = (distance<multiply*R)


        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        force_change_dir = 0

        robot_number = len(distance_local)




        for i in range(len(x_dist)):

            #storing current position 
            current_position = np.array([position[0,i],position[1,i]])


            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])

            #checking if in the same cluster, if so adding to cluster count
            if(mag<0.2*R and mag!=0):
                cluster_count = cluster_count+1
    
    
    cluster_avg = cluster_total/robot_number
    cluster_average.append(cluster_avg)    
   
   
    min_neighbours.append(avg_min_distance)
    interception_time.append(sum(interception_time)+collision_time)

In [ ]:
#Function to create random walk
def random_walk(swarm):
        """
        Updates the movement of the robot by checking if the current heading needs to be reset, and then updating the position           according to the header

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None
        """
        weighting = 20
        deviation = 5
    
        for robot in swarm:
            if robot.check_walk():
                walk_time = int(np.random.normal(weighting,deviation,1))
                heading = random.uniform(0,2*math.pi)
                robot.walk_reset(walk_time,heading)
            x_change = math.cos(robot.heading)*robot.x_velocity
            y_change = math.sin(robot.heading)*robot.y_velocity
            x_updated = robot.x + x_change
            y_updated = robot.y + y_change
            #self.bounce(x_updated,y_updated,x_change,y_change,60000)
            robot.update_position(x_updated,y_updated)
            robot.update_timer()

In [ ]:
#Function to create position change for robots
def update_robots(swarm,G,power,R,max_force,multiply,cluster_average):
    '''
    Updates the robot position based on a chosen algorithm

            Parameters:
                   None
            
            Returns:
                   None

    '''
    #random_walk(swarm)
    physics_walk(swarm,G,power,R,max_force,multiply,min_neighbours,cluster_average)
    return min_neighbour

        
        


In [ ]:
#Function to update/create dust devils
def dust(dust_devils,probability_dust,side,timer,dust_speed):
    '''
    Adds randomly generated dust devil object to list and pops the dust devils when their time is up

            Parameters:
                    


    '''
    update_dust(dust_devils,dust_speed)
    generated = random.uniform(0,1)
    amount = len(dust_devils)
    #checking if a dust devil should be generated
    if(generated<probability_dust):
        #generating a random number between 1 and 999 which is within the maximum domain of x and y
        x = random.randint(-side*1000,side*1000)
        y = random.randint(-side*1000,side*1000)
        radius = random.randint(0,500)
        x_trajectory,y_trajectory = trajectory_dust(x,y,dust_speed)
        #adding to the dust devils list
        print("Dust Devil Added: ", [x,y])
        dust_devils.append(DustDevil(x,y,radius,dust_speed,timer,dust_time,x_trajectory,y_trajectory))
    
    i = 0
    while i<len(dust_devils):
        if(dust_devils[i].end_time < timer ):
            print("Dust Devil Popper: ", [dust_devils[i].x,dust_devils[i].y])
            dust_devils.pop(i)
            i-=1
        i+=1

In [ ]:
#trajectory of the dust devil
def trajectory_dust(x,y,dust_speed):
    x = sym.Symbol('x')
    y = sym.Symbol('y')
    r = dust_speed
    #case = random.uniform(0,2)
    case = 0
    heading = random.uniform(0,2*math.pi)
    if(case == 0):
        x_trajectory = x + r*math.cos(heading)
        y_trajectory = y + r*math.sin(heading)
    elif(case == 1):
        x_trajectory = x 
        y_trajectory = y
    else:
        x_trajectory = x
        y_trajectory = y
    
    return x_trajectory,y_trajectory

In [ ]:
#function to update dust devil positions based on trajectory 
def update_dust(dust_devils,dust_speed):
    x = sym.Symbol('x')
    y = sym.Symbol('y')
    for dust in dust_devils:
        #print("Old X Position: ", dust.x)
        #print("Old Y Position: ", dust.y)
        x_trajectory = dust.x_trajectory
        y_trajectory = dust.y_trajectory
        x_function = lambdify(x, x_trajectory, 'numpy')
        y_function = lambdify(y, y_trajectory, 'numpy')
        x_updated = x_function(dust.x)
        y_updated = y_function(dust.y)
        #print("New X Position: ", x_updated)
        #print("New Y Position: ", y_updated)
        dust.update_position(x_updated,y_updated)

In [ ]:
swarm = initialise(robot_number, side)

In [ ]:
#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [ ]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    
    physics_walk(swarm,G,power,R,max_force,multiply,min_neighbours,cluster_average,dust_devils,detection_range,interception_time)
    x_new,y_new = positions(swarm)
    #creating scatter plot of robots
    data = go.Scatter(
        x=x_new,
        y=y_new,
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-1000, 1000]),xaxis = dict(range=[-1000,1000]))
        }
    )
    if(timer>1000):
        dust(dust_devils,probability_dust,side,j,dust_speed)


    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment </b>",
                      
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 1
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    if(n>1):
        min_neighbour = min_neighbours[-1]
    else:
        min_neighbour = 0
    fig = go.Figure(data=[go.Table(header=dict(values=['<b>Simulation Parameters</b>','<b>Values</b>'],
                            line_color='black',
                            font=dict(color='black', size=17)),
                            cells=dict(values=[[ '<b>Simulation Timestep (h:min:s)</b>','<b>Number of Robots</b>','<b>Sensor Range (m)</b>', '<b>Gravitational Constant</b>','<b>Power</b>','<b>Local Multiplier</b>','<b>Max Force (N)</b>', '<b>Max Speed (m/s)</b>', '<b>Minimum Neighbour Average (m)</b>', ], [str(datetime.timedelta(seconds=timer)),robot_number,R,G,power,multiply,max_force,robot_speed,min_neighbour]],align='center',line_color='black'))])    #fig.layout['template']['data']['table'][0]['header']['fill']['color']='#636EFA'
    fig.update_layout(width = 650, height = 700,margin=dict(
t=260, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.3')

In [ ]:
x = np.array([0,2,3])
print()

In [ ]:
print(min_neighbours)

In [ ]:
        performance = Processing_Functions.performance_graph(min_neighbours,"","Time (s)","Minimum Average Neighbour Distance (m)")
        performance.write_image("" + "- Performance.png")

In [ ]:
        performance = Processing_Functions.performance_graph(interception_time,"","Time (s)","Minimum Average Neighbour Distance (m)")
        performance.write_image("" + "- Performance.png")

In [ ]:
print(interception_time)